In [1]:
%pylab inline
import pandas as pd
import numpy as np
import scipy as sp
from pandas.io.json import json_normalize
import json
from datetime import date, timedelta
from time import time

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore');

Populating the interactive namespace from numpy and matplotlib


In [2]:
df_raw = pd.read_csv('/Users/bwu/testing/large_data_folder/last_carousel_table_5_rolling.csv')

In [3]:
#Initial cleaning of feature names
df_raw.columns = [col_names.replace('rolling_','') for col_names in df_raw.columns.values]

In [4]:
df_raw.head()

,first_visit_num,experiment_visid,experiment_control,experiment_test,visid,visit_num,visit_ux_type,visit_date,visit_hour,visit_geo_country,...,past28d_page_interactions_per_view,past28d_views_section_max,past28d_views_property_max,past28d_visits_ref_max,past28d_visits_camp_max,past28d_views_section_primary,past28d_views_property_primary,past28d_visits_ref_primary,past28d_visits_camp_primary,visit_date_num
0,1,100773303390494114,0,1,100773303390494114,2,Desktop Web,2016-08-21 00:00:00 UTC,12,usa,...,0.25000,1.0,0.0,1.0,1.0,LDP,NaN,Search,Other,1
1,1,1008930168569201832,0,1,1008930168569201832,31,Desktop Web,2016-08-21 00:00:00 UTC,16,usa,...,0.35533,158.0,176.0,10.0,5.0,SRP,Buy,Direct,NaN,1
2,1,1014712710176659876,1,0,1014712710176659876,1,Desktop Web,2016-08-30 00:00:00 UTC,8,usa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,1,1014866083519326140,1,0,1014866083519326140,40,Desktop Web,2016-08-22 00:00:00 UTC,4,usa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,1,10154653262486997040,0,1,10154653262486997040,24,Desktop Web,2016-08-23 00:00:00 UTC,8,usa,...,0.45000,200.0,242.0,8.0,8.0,SRP,Buy,Direct,Other,1


In [5]:
#Desktop only
df_raw = df_raw[df_raw.visit_ux_type=="Desktop Web"]

### Select Relavant Features

In [6]:
# seller = ['visit_nfs_recentlysold_lead', 'current_day_nfs_recentlysold_lead', 'past28d_nfs_recentlysold_lead', 
#           'past7d_nfs_recentlysold_lead','visit_seller_lead', 'current_day_seller_lead', 'past28d_seller_lead', 'past7d_seller_lead',
#             'visit_nfs_offmarket_lead', 'current_day_nfs_offmarket_lead', 
#             'past28d_nfs_offmarket_lead', 'past7d_nfs_offmarket_lead', 
#             'planning_to_sell', 'registered_seller', 'new_survey_seller', ]

In [7]:
identifiers = ['experiment_control', 'experiment_test', 'visid',]

In [8]:
buyer_leads = ['current_day_lead_buy', 
#          'visit_lead_buy', 
#          'past28d_lead_buy', 
#          'past7d_lead_buy'
        ]

In [12]:
secondary_features = ['visit_geo_country', 'visit_geo_region', 'visit_campaign_subtype',
                        'past7d_visits_geo_usa', 
                        'past28d_views_pct_section_home', 'past28d_views_pct_section_srp', 
                        'past28d_views_pct_section_ldp', 'past28d_views_pct_section_news', 
                        'past28d_views_pct_section_acct', 'past28d_views_pct_property_sold', 
                        'past28d_views_pct_property_buy', 'past28d_views_pct_property_rent', 
                        'past28d_views_pct_property_sell', 'past28d_visits_pct_geo_usa', 'past28d_visits_pct_ref_direct', 
                        'past28d_visits_pct_ref_search', 'past28d_visits_pct_ref_internal', 
                        'past28d_visits_pct_ref_social', 'past28d_visits_pct_ref_other', 
                        'past28d_visits_pct_camp_paid_search', 'past28d_visits_pct_camp_email', 
                        'past28d_visits_pct_camp_syndication', 'past28d_visits_pct_camp_display', 
                        'past28d_visits_pct_camp_affiliate', 'past28d_visits_pct_camp_social', 
                        'past28d_visits_pct_camp_other', 'past28d_visits_geo_usa', 
                      
                        'ldp_sale_views', 'ldp_rent_views', 'ldp_unknown_views', 'ldp_sale_price_count', 'ldp_rent_price_count', 'geo_count', 
                        'ldp_sale_price_mean', 'ldp_rent_price_mean', 'ldp_sale_price_median', 'ldp_sale_price_sd', 'ldp_sale_price_cv', 
                        'ldp_rent_price_sd', 'ldp_rent_price_cv', 'latitude_mean', 'longitude_mean', 'geo_distance_to_mean_avg', 
                        'zip_med_income_median', 'zip_med_age_median', 'ldp_sale_price_zip_mean_abs_dev_mean', 
                        'ldp_sale_price_zip_zscore_mean', 'distinct_zips', 'distinct_states',
                      
                        'past7d_ldp_rent_price_count', 'past7d_ldp_sale_price_mean', 'past7d_ldp_rent_price_mean', 
                        'past7d_ldp_sale_price_median', 'past7d_ldp_rent_price_median', 'past7d_ldp_sale_price_sd', 'past7d_zip_med_age_median', 
                        'past7d_ldp_sale_price_cv', 'past7d_ldp_rent_price_sd', 'past7d_ldp_rent_price_cv', 'past7d_latitude_mean', 
                        'past7d_longitude_mean', 'past7d_zip_med_income_median', 'past7d_ldp_sale_price_zip_mean_abs_dev_mean', 'past7d_ldp_sale_price_zip_zscore_mean',

                        'past28d_ldp_sale_price_mean', 'past28d_ldp_rent_price_mean', 'past28d_ldp_sale_price_median',
                        'past28d_ldp_rent_price_median', 'past28d_ldp_sale_price_sd', 'past28d_ldp_sale_price_cv', 
                        'past28d_ldp_rent_price_sd', 'past28d_ldp_rent_price_cv', 'past28d_latitude_mean', 
                        'past28d_longitude_mean',  'past28d_zip_med_income_median', 
                        'past28d_zip_med_age_median',
                        'past28d_ldp_sale_price_zip_mean_abs_dev_mean', 'past28d_ldp_sale_price_zip_zscore_mean', 
                        
                        'visit_date_num', 'visit_date', 
                        'visit_views', 'visit_views_ldp_for_sale', 'visit_views_ldp_rent', 
                     ]

In [13]:
primary_features = ['visit_num', 'visit_hour', 'visit_referrer_type', 
                    'visit_campaign_type', 'visit_landing_page_type', 
                                        
                    'past28d_visits_registered', 
                    
                    'past7d_views', 'past7d_page_interactions', 'past7d_visits', 'past7d_active_days', 
                    'past7d_views_section_home', 'past7d_views_section_srp', 'past7d_views_section_ldp', 
                    'past7d_views_section_news', 'past7d_views_section_acct', 'past7d_views_property_sold', 
                    'past7d_views_property_buy', 'past7d_views_property_rent', 'past7d_views_property_sell', 
                    'past7d_visits_ref_direct', 'past7d_visits_ref_search', 'past7d_visits_ref_internal', 
                    'past7d_visits_ref_social', 'past7d_visits_ref_other', 'past7d_visits_camp_paid_search', 
                    'past7d_visits_camp_email', 'past7d_visits_camp_syndication', 'past7d_visits_camp_display', 
                    'past7d_visits_camp_affiliate', 'past7d_visits_camp_social', 'past7d_visits_camp_other', 
                    
                    'past28d_views', 'past28d_page_interactions', 'past28d_visits', 'past28d_active_days', 
                    'past28d_views_section_home', 'past28d_views_section_srp', 'past28d_views_section_ldp', 
                    'past28d_views_section_news', 'past28d_views_section_acct', 'past28d_views_property_sold', 
                    'past28d_views_property_buy', 'past28d_views_property_rent', 'past28d_views_property_sell', 
                    'past28d_visits_ref_direct', 'past28d_visits_ref_search', 
                    'past28d_visits_ref_internal', 'past28d_visits_ref_social', 'past28d_visits_ref_other', 
                    'past28d_visits_camp_paid_search', 'past28d_visits_camp_email', 'past28d_visits_camp_syndication', 
                    'past28d_visits_camp_display', 'past28d_visits_camp_affiliate', 
                    'past28d_visits_camp_social', 'past28d_visits_camp_other',  
                    
                    'past7d_ldp_sale_views', 'past7d_ldp_rent_views', 'past7d_ldp_unknown_views', 
                    'past7d_ldp_sale_price_count', 'past7d_distinct_zips', 'past7d_distinct_states', 
                    
                    'past28d_ldp_sale_views', 'past28d_ldp_rent_views', 'past28d_ldp_unknown_views', 
                    'past28d_ldp_sale_price_count', 'past28d_ldp_rent_price_count', 
                    'past28d_distinct_zips', 'past28d_distinct_states', 
                                                            
                    'past7d28d_views_ratio', 'past7d_visits_per_active_day', 
                    'past7d_views_per_active_day', 'past7d_views_per_visit', 'past7d_page_interactions_per_view', 
                    
                    'past28d_visits_per_active_day', 'past28d_views_per_active_day', 'past28d_views_per_visit', 
                    'past28d_page_interactions_per_view', 'past28d_views_section_max', 'past28d_views_property_max', 
                    'past28d_visits_ref_max', 'past28d_visits_camp_max', 'past28d_views_section_primary', 
                    'past28d_views_property_primary', 'past28d_visits_ref_primary', 'past28d_visits_camp_primary', 
                   ]

In [25]:
#Only use primary features
df_primary = df_raw[primary_features]

In [26]:
#Seperate Numeric and Categorical features
primary_numeric_columns = df_primary.select_dtypes(include=[np.number]).columns.values
primary_categorical_columns = list(set(df_primary.columns.values) - set(primary_numeric_columns))

### Dummify Categorical Features

In [27]:
primary_categorical_columns

['past28d_visits_camp_primary',
 'visit_campaign_type',
 'visit_referrer_type',
 'past28d_visits_ref_primary',
 'visit_landing_page_type',
 'past28d_views_section_primary',
 'past28d_views_property_primary']

In [28]:
#Other numeric features that should be considered as categorical variables
other_categorical_columns = ['visit_hour']

In [29]:
#Dummify categorical variables
for column in primary_categorical_columns+other_categorical_columns:
    dummies = pd.get_dummies(df_primary[column], prefix = column)
    #Add dummy columns to matrix and remove original column from matrix
    df_primary[dummies.columns] = dummies
    df_primary = df_primary.drop(column,1)

In [34]:
dummified_categorical_features = list(set(df_primary.columns.values) - set(primary_numeric_columns))

### Handle Missing Numeric Values

In [39]:
#Change all numeric missing values to zeroes (this is in fact true for this dataset - nan represent 0's)
df_primary = df_primary.fillna(0)

In [44]:
#Check if there are any null values remaining in the data
pd.isnull(df_primary).sum().sum()

0

In [45]:
df_primary.head()

,visit_num,past28d_visits_registered,past7d_views,past7d_page_interactions,past7d_visits,past7d_active_days,past7d_views_section_home,past7d_views_section_srp,past7d_views_section_ldp,past7d_views_section_news,...,visit_hour_14,visit_hour_15,visit_hour_16,visit_hour_17,visit_hour_18,visit_hour_19,visit_hour_20,visit_hour_21,visit_hour_22,visit_hour_23
0,2,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,31,5.0,129.0,35.0,5.0,3,5.0,112.0,9.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,40,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24,2.0,40.0,34.0,1.0,1,1.0,34.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Standardize Numeric Features

### Remove Numeric Outliers

In [46]:
def is_outlier_mean(data, m = 3):
    return abs(data - np.mean(data)) > m * np.std(data)

In [ ]:
rows_with_outliers = []
for col in dropped_categorical_df.columns[:-1]:
    rows_with_outliers.append(np.where(is_outlier_mean(raw_df[col])))
    
outlier_rows = list(unique([item for sublist in [row[0] for row in rows_with_outliers] for item in sublist]))


In [ ]:
removed_outliers_df = pd.DataFrame()
removed_outliers_df = dropped_categorical_df.drop(dropped_categorical_df.index[outlier_rows])

### Lift and Segement Size Function for Numeric Features